<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Using-bigger-data" data-toc-modified-id="Using-bigger-data-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Using bigger data</a></span><ul class="toc-item"><li><span><a href="#Training-data" data-toc-modified-id="Training-data-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Training data</a></span></li></ul></li></ul></div>

In [31]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [32]:
ls ..

__init__.py   data/         notebooks/    pyngramspell/


In [33]:
import os,sys,inspect
currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parentdir = os.path.dirname(currentdir)
sys.path.insert(0,parentdir) 

In [34]:
import data
from data import utils

train, test = utils.get_evaluation()

X_tr = [x['original'] for x in train]
y_tr = [x['corrected'] for x in train]
X_te = [x['original'] for x in test]
y_te = [x['corrected'] for x in test]

train_data = [' '.join(x) for x in y_tr]

In [35]:
import pyngramspell
speller = pyngramspell.PyNgramSpell()

In [36]:
speller.fit(train_data)

In [37]:
speller.transform('the maan went to the store')

['the', 'man', 'went', 'to', 'the', 'store']

In [38]:
speller.transform('red shirtt')

['red', 'shirtt']

In [40]:
%%time 
speller.transform('red shirtt')

CPU times: user 261 µs, sys: 2 µs, total: 263 µs
Wall time: 265 µs


['red', 'shirtt']

### Using bigger data

#### Training data


The dataset used to train the model comes from  https://jmcauley.ucsd.edu/data/amazon/

The `Video_Games_5.json` can be downloaded from 
http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/reviews_Video_Games_5.json.gz

In [15]:
import pandas as pd
from os.path import expanduser
home = expanduser("~")

In [16]:
dataset_path = os.path.join(home,'Datasets/amazon_product_data/reviews_Video_Games_5.json')
df = pd.read_json(dataset_path, lines=True)

In [17]:
df.head()

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime
0,A2HD75EMZR8QLN,0700099867,123,"[8, 12]",Installing the game was a struggle (because of...,1,Pay to unlock content? I don't think so.,1341792000,"07 9, 2012"
1,A3UR8NLLY1ZHCX,0700099867,"Alejandro Henao ""Electronic Junky""","[0, 0]",If you like rally cars get this game you will ...,4,Good rally game,1372550400,"06 30, 2013"
2,A1INA0F5CWW3J4,0700099867,"Amazon Shopper ""Mr.Repsol""","[0, 0]",1st shipment received a book instead of the ga...,1,Wrong key,1403913600,"06 28, 2014"
3,A1DLMTOTHQ4AST,0700099867,ampgreen,"[7, 10]","I got this version instead of the PS3 version,...",3,"awesome game, if it did not crash frequently !!",1315958400,"09 14, 2011"
4,A361M14PU2GUEG,0700099867,"Angry Ryan ""Ryan A. Forrest""","[2, 2]",I had Dirt 2 on Xbox 360 and it was an okay ga...,4,DIRT 3,1308009600,"06 14, 2011"


In [18]:
train_data = list(df['reviewText'])

In [19]:
import pyngramspell
speller = pyngramspell.PyNgramSpell()

In [20]:
speller.fit(train_data)

In [21]:
len(speller.vocabulary)

48316

In [27]:
%%timeit
speller.transform('xboj 360 colector edition')

17.7 ms ± 3.25 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [28]:
%%timeit
speller.transform('pssp blac edition')

13.3 ms ± 339 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [26]:
%%timeit
speller.transform('new pleystation gaime batmen')

17.5 ms ± 807 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [416]:
'batmen' in speller.vocabulary

True

In [418]:
speller.unigram_freq_dict['batmen']

1

Because sometimes very infrequent errors are inside the vocabulary we might want to remove them

In [420]:
speller.filter_vocabulary(min_freq=5)

In [421]:
'batmen' in speller.vocabulary

False

In [422]:
speller.transform('new pleystation gaime batmen')

['new', 'playstation', 'game', 'batman']